# TRPO and PPO

- <font color=blue>**从Natural PG到TRPO，再到PPO的solution**</font>：
  1. vanilla policy gradient使用的迭代式训练的效果非常不稳定，
  2. Natrual PG从policy iteration出发，推导出了带KL divergence约束的<font color=blue>**Importance Sampling形态的**</font>Policy Gradient优化目标，但是求解的方法相对复杂。
  3. TRPO使用共轭梯度简化了Natural PG的优化目标。
  4. 而PPO为了充分利用深度学习现成的工具，从梯度迭代式反过来思考。找到了与迭代式对应的surrogate loss来构建优化问题。并在此基础上进一步优化了step size的设计思路。

- <font color=brown>[notes of UCB RL bootcamp(2021) lec5]</font>

## I. TRPO

### I.1 出发点

#### I.1.1 背景知识：RL和DL样本分布的差异

- 一般的DL中样本是iid的，也就是说通常可以假设样本都是来自同一个分布。但是这个假设在RL中不成立。
- RL中有两个因素导致iid不成立：
  1. 如果策略稳定，并且状态s的分布达到了稳定的d(s)。按照episodes来收集数据集，每个episode(trajectory)上，连续的time-steps中，前一步的状态必然影响后一步的状态，所以每个step上收集到的数据是相关的。
  2. 在算法迭代过程中，策略在不断更新，所以状态s是non-stationary distribution，基本不会处于稳定的d(s)上。也就是说，收集到的数据并不是来自相同的分布。这里还有一个因素让策略的更新过程并不smooth，也就是一次迭代产生的两个策略之间差别可能很大：
     - 算法迭代的参数是$\pi(\theta)中的\theta$，<font color=orange>NN中可以控制步幅让DL中的参数迭代量很小，这只能让参数的变化幅度小，但是从结果来看，策略本身的变化却可能很大。</font>
- 上述RL中数据分布的相关性和不稳定性会让RL算法迭代过程中reward波动很大。常见的现象是，迭代过程中很长时间reward都很小，然后过了很久才突然有一个很大的提升。它不会像supervised DL中，指标会逐步稳定上升。

#### I.1.2 出发点1：AC算法的两大问题

  1. 很难选择stepsize：这个问题在RL中比在深度学习中带来的后果更糟糕。
     - 由于数据分布不稳定，很难像DL中那样找到一个大小合适的stepsize，并且在整个训练过程中几乎不用太调整这个stepsize。在DL中，可以用normalization的方法来稳定让网络中的变量分布，然后用更好的优化器（如Adam）来拟合二阶优化，达到控制参数不同维度的步幅效果。这样就能让迭代过程稳定。但如前文分析，即使这样做也只能稳定网络参数的变化，却无法稳定策略本身，所以需要有办法稳定策略本身的变化。
     - 而且错误的stepsize带来的后果也更糟糕。如果stepsize略大，policy gradient得到的策略可能是一个非常差的策略。和DL中不一样，此时新的数据就会是用这个糟糕的策略生成的，这会进入无法恢复的恶性循环。
  2. sample efficiency很低，同样因为是on-policy算法，一个batch的trajectory用一次就扔掉。

- **TRPO提出的solution**：
  1. 用trust region来减缓stepsize的问题。trust region不是控制参数$\theta$的变化幅度，而是直接控制一次迭代导致的策略变化差异。这就针对性的解决了前文中提到的策略波动导致的问题。
  2. 更有效地使用样本，尽管仍然是on-policy algorithm，仍然用一次就扔掉，但样本使用效率提高了。
     - trust region让得到的糟糕样本更少

#### I.1.3 出发点2：参考DL，将RL处理成优化问题

- 深度学习中，learning问题被转化为numerical optimization problem。
  - 典型例子是：Supervised learning的优化目标就是minimize training error
- 强化学习的目标是：如何利用所有搜集到的data来找到最优策略？
  - <font color=blue>**Q-learning** </font>：通过off-policy做到了**能够利用所有截至目前所有搜集到的data**，但是优化的目标是Bellman Error，不是真正的目标：best policy
  - <font color=blue>**vanilla policy gradient**</font>：用了stochastic gradient，但是并没有将问题转化为优化问题。<font color=orange>[rk's note]: </font>PG的目标是$max U(\theta)$，用TD方法来求解，理论上这也是优化问题。但是实际求解过程中，经过rewards to go和增加baseline等操作，最终得到的算法已经没有一个对应的优化目标了。而John Schulman这里想做的，是像DL中一样，以一个loss function为目标，通过直接求目标梯度来求解最优策略。<font color=green>**这还有一个非常好的优点，就是所有的DL framework等工具就能直接用在RL算法上了。**</font>

### I.2 surrogate loss

#### I.2.1 以vanilla PG为迭代式的loss

- PG迭代式：
$$\hat g = \hat E_t \left[ \nabla_{\theta}log\pi_{\theta} (a_t|s_t)\hat A_t \right]$$
  - <font color=orange>E表示多个time-steps的均值的估计</font>

- 直接与之对应的loss形式：
$$max L^{PG} (\theta)= \hat E_t \left[ log\pi_{\theta} (a_t|s_t)\hat A_t \right]$$

- 这个loss形式实践中不会直接使用，因为：
  - 当$\hat A_t>0$时，用这个式子为目标会让$\pi(a_t|s_t)$增加，往1走；反之，当$\hat A_t<0$时，用这个式子为目标会让$\pi(a_t|s_t)$减少，往0走。
  - 但实际迭代过程中有一个问题，$\hat A_t$的估计是非常noisy的。当noisy导致advantage的符号发生频繁改变，此时梯度的variance很大，基于advantage的估计量调整的策略会发生radically change。
  - 如果要用的话，一定要很大的batch size，但调参特别是调learning rate会很难，用adaptive step size的优化器比如adam，普通优化器比如SGD更难work。

#### I.2.2 另一种仍以policy gradient为迭代式的loss

- <font color=green>**思路：先将梯度转换成一种等价的梯度形式，然后再用该等价的梯度形式构造loss函数。**</font>

- 两种等价的梯度形式：
$$\nabla_{\theta}log f(\theta)|_{\theta=\bar \theta}=\frac{\nabla_{\theta}f(\theta)|_{\theta=\bar \theta}}{f(\bar \theta)} = \nabla_{\theta}\left(\frac{f(\theta)}{f(\bar \theta)}\right)|_{\theta=\bar \theta}$$

- 从上式可知，policy gradient的迭代式可以转换为等价的梯度迭代式：

$$\begin{align}
\hat g & = \hat E_t \left[ \nabla_{\theta}log\pi_{\theta} (a_t|s_t)\hat A_t \right] \\
& = \sum_{t=0}^{H-1}\left( \nabla_{\theta }log\pi_{\theta}(a_t|s_t)\hat A_t |{\color{Red} {\theta_t}}  \right )\\
& = \sum_{t=0}^{H-1}\left( \frac{\nabla_{\theta }\pi_{\theta}(a_t|s_t) |{\color{Red} {\theta_t} }}{\pi_{{\color{Red} {\theta_t} }}(a_t|s_t)}{\color{Red} {\hat A_t}}  \right )\\
&  = \hat E_t \left[ \frac{\nabla_{\theta }\pi_{\theta}(a_t|s_t) |{\color{Red} {\theta_t}} }{\pi_{{\color{Red} {\theta_t} }}(a_t|s_t)}{\color{Red} {\hat A_t}}  \right] \\
\end{align}$$

- 上式对应的loss形态：
$$L(\theta )= \hat E_t \left[ \frac{\pi_{\theta}(a_t|s_t)  }{\pi_{\color{Red} {\theta_t} }(a_t|s_t)}{\color{Red} {\hat A_t}}  \right]$$

#### I.2.3 importance sampling的要点

- 思路：如果没办法直接从 $p(x)$ 中采样数据，可以从 $q(x)$ 中采样数据，$x^i\sim q(x)$
$$
\begin{split}
E_{x\sim p}[f(x)]&=\int f(x)p(x)dx=\int f(x)\underbrace{\frac{p(x)}{q(x)}}_{\text{importance weight}}q(x)dx\\
&=E_{x\sim q}[f(x)\frac{p(x)}{q(x)}]
\end{split}
$$

- 期望相同
$$E_{x\sim p}[f(x)]=E_{x\sim q}[f(x)\frac{p(x)}{q(x)}]$$

- 方差不同： q的分布与p的分布越接近，方差越小。如果两个分布差异很大，则估计的结果方差大，也就是说大多数时候估计得到的结果偏差很大。
$$\begin{split}
    \text{Var}_{x\sim q}[f(x)\frac{p(x)}{q(x)}]&=E_{x\sim q}[(f(x)\frac{p(x)}{q(x)})^2]-(E_{x\sim q}[f(x)\frac{p(x)}{q(x)}])^2\\
    &=E_{x\sim p}[f^2(x)\frac{p(x)}{q(x)}] - (E_{x\sim p}[f(x)])^2
\end{split}$$

- 在梯度上使用importance sampling的例子：
$$\begin{align}
\nabla U(\theta )& = \nabla \underset{\tau\sim \pi _\theta}{E} [R(\tau)]\\
& = \nabla \underset{\tau\sim \pi_{\theta_{old}}}{E} \ \  [\frac{P_{\theta}(\tau)}{P_{\theta_{old}}(\tau)}R(\tau)]\\
& =  \underset{\tau\sim \pi_{\theta_{old}}}{E} [\frac{\nabla P_{\theta}(\tau)}{P_{\theta_{old}}(\tau)}R(\tau)]\\
\end{align}$$
  - <font color=red>但这样直接用importance sampling的话，梯度展开难以求解。</font>

#### I.2.3 理解surrogate loss

$$L^{IS}(\theta )= \hat E_t \left[ \frac{\pi_{\theta}(a_t|s_t)  }{\pi_{{\color{Red} {\theta_t}} }(a_t|s_t)}{\color{Red} {\hat A_t}}  \right] $$

- <font color=norange>**从importance sampling角度理解这个loss形态：**</font>
  - 所有作为样本点的state-action pair都是用$\pi_{\theta_t}$策略获得的，advantage也是用这些data估计的。
  - 因此，从IS的角度来看，这个loss是用 $\pi_{\theta_t}$ 时的策略分布在估计Advantage在策略为$\pi(\theta)$时的均值。

- Importance sampling角度重新拆解该loss：

$$

### I.3 TRPO算法

- surrogate loss + stepsize constraint

$$\begin{align}
& \underset{\theta }{max} \ \hat E_t \left[ \frac{\pi_{\theta}(a_t|s_t)  }{\pi_{ \theta_{old}}(a_t|s_t)} \hat A_t  \right] \\
& s.t. \ \hat E_t [KL(\pi_{\theta_{old}}(·|s_t) |\pi_{\theta}(·|s_t) )] \le \delta 
\end{align}$$

- 求解优化问题的方法：
  - 要用到conjugate gradient，其中涉及Hessian matrix的逆，复杂度较高。[略]

- 算法伪码：<font color=brown>[ref: spinning up]</font>
  - <img src='pics/trpo.png' width='70%'>

## II. PPO

### II.1 出发点

- TRPO算法仍然存在的问题：
  1. trust region constraints很难直接融入neural network的设计
  2. 为了计算二阶梯度条件而使用的conjugate gradient计算起来很复杂
  - John Schulman当时想找到一种不需要二阶梯度的方法来求解TRPO中的优化问题

- PPO的solution：将TRPO中的优化问题转化为等价的没有constraint的优化问题

### II.2 目标函数形式优化

#### II.2.1 有约束优化问题直接转无约束优化问题

- 将TRPO中的有约束的优化问题转化为无约束优化问题：
$$\begin{align}
& \underset{\theta }{max} \ \hat E_t \left[ \frac{\pi_{\theta}(a_t|s_t)  }{\pi_{ \theta_{old}}(a_t|s_t)} \hat A_t  \right] \\
& s.t. \ \hat E_t [KL(\pi_{\theta_{old}}(·|s_t) |\pi_{\theta}(·|s_t) )] \le \delta  \\
等价问题：\\
& \underset{\theta }{max} \ \hat E_t \left[ \frac{\pi_{\theta}(a_t|s_t)  }{\pi_{ \theta_{old}}(a_t|s_t)} \hat A_t - \beta KL(\pi_{\theta_{old}}(·|s_t) |\pi_{\theta}(·|s_t) )\right]  
\end{align}$$

- 带来新的问题：
  1. 虽然达到了转为无约束优化问题的目标，现在可以直接使用DL的方法解RL问题。但$\beta$很难取到合适的值。
  2. 这个形式下，无法达到TRPO迭代中的一个优势：monotonic improvement。

#### II.2.2 将优化问题改为Clip形式

$$\begin{align}
r_t(\theta ) & = \frac{\pi_{\theta}(a_t|s_t)  }{\pi_{ \theta_{old}}(a_t|s_t)} \\
L^{CLIP}(\theta ) & = \hat E_t \left[ min[r_t(\theta )\hat A_t, clip(r_t(\theta ), 1-\epsilon, 1+\epsilon )\hat A_t]\right] \\ 
\end{align}$$

- 一种更好理解的写法：
$$\begin{align}
& L^{CLIP}(\theta )  = \hat E_t \left[ min[r_t(\theta )\hat A_t, g(\epsilon ,\hat A_t)]\right] \\
& g(\epsilon ,\hat A_t) = \left\{\begin{matrix}
 (1-\epsilon ) \hat A_t& A\ge 0\\
 (1+\epsilon ) \hat A_t & A\le 0
\end{matrix}\right. , \ \ \ r_t(\theta ) = \frac{\pi_{\theta}(a_t|s_t)  }{\pi_{ \theta_{old}}(a_t|s_t)} \\
\end{align}$$

#### II.2.3 Clip目标函数形式的理解

1. $clip(r_t(\theta ), 1-\epsilon, 1+\epsilon )\hat A_t$的作用：当新旧策略一模一样的时候，r值为1，而clip截断新旧策略分布的比例，使它保持在$(1-\epsilon, 1+\epsilon)$范围内。实际上就是让新策略相对旧策略变化不要太大。
   - 实现方式：如果$A>0，r>1+\epsilon$，此时clip part取值为$(1+\epsilon)A$

？？？

### II.3 算法

- 算法伪码：<font color=brown>[ref: spinning up]</font>
  - <img src='pics/ppo.png' width='70%'>